# Tutorial: Creating a list of candidate Collecitve Variable susing GeneratePlumed()

In [1]:
from plumed import GeneratePlumed

### Adding distance, angle, or torsion CVs

In [2]:
x = GeneratePlumed() # Initiate class

# Add Collective Variables (CV's) or Order Parameters (OP's)

x.add_distances(atoms=[1,2,4], diff_of_dist=True, periodic=False, components=False)
x.add_angles(atoms=[1,2,3,4], periodic=False)
x.add_torsion(atoms=[1,3,4,5], periodic=False)

# The defaults for perioidc and components are False, alternatively you could use: 

    #x.add_distances(atoms=[1,2,4], diff_of_dist=True)
    #x.add_angles(atoms=[1,2,4])
    #x.add_torsion(atoms=[1,2,3,4])

print(x.string)


# DISTANCES 
d0: DISTANCE ATOMS=1,2 NOPBC
d1: DISTANCE ATOMS=1,4 NOPBC
d2: DISTANCE ATOMS=2,4 NOPBC

# DIFFERENCE OF DISTANCES 
dd0: COMBINE ARG=d0,d1 COEFFICIENTS=1.0,-1.0 PARAMETERS=0.0,0.0 POWERS=1.0,1.0 PERIODIC=NO
dd1: COMBINE ARG=d0,d2 COEFFICIENTS=1.0,-1.0 PARAMETERS=0.0,0.0 POWERS=1.0,1.0 PERIODIC=NO
dd2: COMBINE ARG=d1,d2 COEFFICIENTS=1.0,-1.0 PARAMETERS=0.0,0.0 POWERS=1.0,1.0 PERIODIC=NO

# ANGLES 
a0: ANGLE ATOMS=1,2,3 NOPBC
a1: ANGLE ATOMS=1,2,4 NOPBC
a2: ANGLE ATOMS=1,3,4 NOPBC
a3: ANGLE ATOMS=2,3,4 NOPBC

# TORSION 
t0: TORSION ATOMS=1,3,4,5 NOPBC



### Use add_group function to define more complex CVs

In add_group the atom list is a string, this allows for freedom to define any group of atoms together

In [3]:
# Sometimes it is helpful to create groups of atoms for more complex CVs

x = GeneratePlumed()

# Good for coordination numbers
x.add_group(group_name=['wat','na','cl'], atoms=['1-100:3','1','2'])
x.add_coordination(group_a=['wat','wat','na'], group_b=['na','cl','cl'],
                   r_0=[0.26,0.26,0.15], nl_cutoff=[0.5,0.5,0.5], nlist=True, 
                   nn=6, mm=0, d_0=0.0)

# Or to define puckering coordinates for a sugar ring
x.add_group(group_name=['r1'], atoms=['1,2,3,4,5']) 
x.add_puckering(atoms=[['r1'],[1,2,3,4,5,6]])

print(x.string)


#GROUPS OF ATOMS 
wat: GROUP ATOMS=1-100:3
na: GROUP ATOMS=1
cl: GROUP ATOMS=2

#COORDINATION 
c0: COORDINATION GROUPA=wat GROUPB=na R_0=0.26 NN=6 MM=0 D_0=0.0 NLIST NL_CUTOFF=0.5 NL_STRIDE=1
c1: COORDINATION GROUPA=wat GROUPB=cl R_0=0.26 NN=6 MM=0 D_0=0.0 NLIST NL_CUTOFF=0.5 NL_STRIDE=1
c2: COORDINATION GROUPA=na GROUPB=cl R_0=0.15 NN=6 MM=0 D_0=0.0 NLIST NL_CUTOFF=0.5 NL_STRIDE=1

#GROUPS OF ATOMS 
r1: GROUP ATOMS=1,2,3,4,5

#PUCKERING 
puck0: PUCKERING ATOMS=r1
puck1: PUCKERING ATOMS=1,2,3,4,5,6



### Add basin information, to define stopping criteria for aimless shooting

In [4]:
# Add basin information for aimless shooting 

x = GeneratePlumed()

x.add_distances(atoms=[1,2], diff_of_dist=False)
x.add_committor(basins=2,basin_CV=['d1','d2'],limit=[[0.2,0.1,0.4,0.6],[1.3,1.5,0.3,0.6]])

print(x.string)

Difference of distance CV not generated, requires at least 2 distance CVs

# DISTANCES 
d0: DISTANCE ATOMS=1,2 NOPBC

COMMITTOR ...
ARG=d1,d2
BASIN_LL1=0.2,0.1
BASIN_UL1=0.4,0.6
BASIN_LL2=1.3,1.5
BASIN_UL2=0.3,0.6
STRIDE=1
... COMMITTOR



# Example of generating a plumed file
-> Initiate class  
-> Add candidate CVs  
-> Add a committor (to define basins)  
-> Write to file

In [5]:
x = GeneratePlumed()

x.add_distances(atoms=[1,2,4], diff_of_dist=True)
x.add_angles(atoms=[1,2,4])
x.add_torsion(atoms=[1,2,3,4])

x.add_group(group_name=['wat','na','cl'], atoms=['1-100','1','2']) 
x.add_coordination(group_a=['wat','wat','na'], group_b=['na','cl','cl'],
                   r_0=[0.26,0.26,0.15], nl_cutoff=[0.5,0.5,0.5], nlist=True)

x.add_committor(basins=2,basin_CV=['d1'],limit=[[0.2,0.4],[1.3,1.5]])
    
x.write_file(plumed_file='plumed.dat')

In [6]:
%%script bash
# Preview the file we just created 

cat plumed.dat

# vim:ft=plumed

# DISTANCES 
d0: DISTANCE ATOMS=1,2 NOPBC
d1: DISTANCE ATOMS=1,4 NOPBC
d2: DISTANCE ATOMS=2,4 NOPBC

# DIFFERENCE OF DISTANCES 
dd0: COMBINE ARG=d0,d1 COEFFICIENTS=1.0,-1.0 PARAMETERS=0.0,0.0 POWERS=1.0,1.0 PERIODIC=NO
dd1: COMBINE ARG=d0,d2 COEFFICIENTS=1.0,-1.0 PARAMETERS=0.0,0.0 POWERS=1.0,1.0 PERIODIC=NO
dd2: COMBINE ARG=d1,d2 COEFFICIENTS=1.0,-1.0 PARAMETERS=0.0,0.0 POWERS=1.0,1.0 PERIODIC=NO

# ANGLES 
a0: ANGLE ATOMS=1,2,4 NOPBC

# TORSION 
t0: TORSION ATOMS=1,2,3,4 NOPBC

#GROUPS OF ATOMS 
wat: GROUP ATOMS=1-100
na: GROUP ATOMS=1
cl: GROUP ATOMS=2

#COORDINATION 
c0: COORDINATION GROUPA=wat GROUPB=na R_0=0.26 NN=6 MM=0 D_0=0.0 NLIST NL_CUTOFF=0.5 NL_STRIDE=1
c1: COORDINATION GROUPA=wat GROUPB=cl R_0=0.26 NN=6 MM=0 D_0=0.0 NLIST NL_CUTOFF=0.5 NL_STRIDE=1
c2: COORDINATION GROUPA=na GROUPB=cl R_0=0.15 NN=6 MM=0 D_0=0.0 NLIST NL_CUTOFF=0.5 NL_STRIDE=1

COMMITTOR ...
ARG=d1
BASIN_LL1=0.2
BASIN_UL1=0.4
BASIN_LL2=1.3
BASIN_UL2=1.5
STRIDE=1
... COMMITTOR

PRINT STRIDE=